In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import argparse
import re
import time
from datetime import timedelta
from functools import partial
import logging
from datasets import load_dataset, Dataset
from tqdm import tqdm
import os
from torch.utils.data import DataLoader

from src.retriever import Retriever
from types import SimpleNamespace

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [3]:

arg = SimpleNamespace()
arg.retrieval_model_name_or_path="facebook/contriever-msmarco"
arg.retrieval_embedding_size=768
arg.passages='data/corpus/psgs_w100.tsv'
arg.passages_embeddings='data/corpus/wikipedia_embeddings/*'
arg.indexing_batch_size=1000000
arg.save_or_load_index = False
arg.retrieval_n_subquantizers=0
arg.retrieval_n_bits=8
arg.max_k=100
arg.lowercase = False
arg.normalize_text = False
arg.per_gpu_batch_size=1000000
arg.question_maxlength=100000

retriever = Retriever(arg)
retriever.setup_retriever()

Loading model from: facebook/contriever-msmarco


/home/tkdrnjs0621/miniconda3/envs/torch_241/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Indexing passages from files ['data/corpus/wikipedia_embeddings/passages_00', 'data/corpus/wikipedia_embeddings/passages_01', 'data/corpus/wikipedia_embeddings/passages_02', 'data/corpus/wikipedia_embeddings/passages_03', 'data/corpus/wikipedia_embeddings/passages_04', 'data/corpus/wikipedia_embeddings/passages_05', 'data/corpus/wikipedia_embeddings/passages_06', 'data/corpus/wikipedia_embeddings/passages_07', 'data/corpus/wikipedia_embeddings/passages_08', 'data/corpus/wikipedia_embeddings/passages_09', 'data/corpus/wikipedia_embeddings/passages_10', 'data/corpus/wikipedia_embeddings/passages_11', 'data/corpus/wikipedia_embeddings/passages_12', 'data/corpus/wikipedia_embeddings/passages_13', 'data/corpus/wikipedia_embeddings/passages_14', 'data/corpus/wikipedia_embeddings/passages_15']
Loading file data/corpus/wikipedia_embeddings/passages_00
Total data indexed 1000000
Loading file data/corpus/wikipedia_embeddings/passages_01
Total data indexed 2000000
Loading file data/corpus/wikiped

In [4]:
#retriever.reset_args(arg)

In [5]:
from datasets import Dataset
dataset = Dataset.from_json("data/eval_data/triviaqa_test.jsonl")

In [8]:
idx = 0
print(dataset[idx]['question'])
print(dataset[idx]['answers'])
rs = retriever.search_document(['a'], top_n=10)
rs = retriever.search_document(['a','b'], top_n=10)
rs = retriever.search_document(['a','b','c'], top_n=10)

rs = retriever.search_document(['a'+str(i) for i in range(100)], top_n=10)
print(rs)

Who was the man behind The Chipmunks?
['David Seville', 'david seville']
Questions embeddings shape: torch.Size([1, 768])


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:08<00:00,  8.91s/it]


Search time: 8.9 s.
Questions embeddings shape: torch.Size([2, 768])


100%|██████████| 1/1 [00:09<00:00,  9.89s/it]


Search time: 9.9 s.
Questions embeddings shape: torch.Size([3, 768])


100%|██████████| 1/1 [00:09<00:00,  9.93s/it]


Search time: 9.9 s.
Questions embeddings shape: torch.Size([100, 768])


100%|██████████| 1/1 [00:04<00:00,  4.88s/it]

Search time: 4.9 s.
[{'id': '8146388', 'title': 'A10 autoroute', 'text': "A10 is part of the E-road E05; it is also part of the E50 north of the A11 split near Chartres and the E60 between exit 14 at Orléans and exit 19 at Tours. Most of the A10 is a toll road, but it is free north of the N104, near Paris, between exits 20 and 22 in Tours, and south of the N10 (exit 39), near Bordeaux. Exits are numbered from north to south. A10 autoroute The A10, also called L'Aquitaine, is an Autoroute in France, running for 549 km (341 mi) from the A6 south of Paris to"}, {'id': '3564749', 'title': 'HMS A10', 'text': 'HMS A10 HMS "A10" was an submarine built for the Royal Navy in the first decade of the 20th century. After surviving World War I, she was sold for scrap in 1919. "A10" was a member of the first British class of submarines, although slightly larger, faster and more heavily armed than the lead ship, . The submarine had a length of overall, a beam of and a mean draft of . They displaced o

In [9]:
print(len(rs))

10
